In [52]:
import pandas as pd
from glob import glob
import numpy as np
from easydict import EasyDict as edict
import os
import time
import sys
from tabulate import tabulate

sys.path.insert(0,os.path.expanduser('~/git/torchseg'))
print(sys.path)

from utils.config import load_config
from utils.summary_to_csv import config_to_log,load_log,edict_to_pandas,today_log,recent_log
def summary(rootpath,tags,filter_str=None,recent_log_number=100):
    config_files=glob(os.path.join(rootpath,'**','config.txt'),recursive=True)
    if filter_str == 'today':
        config_files=today_log(config_files)
    elif filter_str == 'recent':
        config_files=recent_log(config_files,recent_log_number)
    elif filter_str is not None:
        config_files=[f for f in config_files if f.find(filter_str)>=0]
    
    tasks=pd.DataFrame()
    for cfg in config_files:
        log=config_to_log(cfg)
        if log is not None:
            ed=load_config(cfg)
            metrics=load_log(log,tags)
            for key,value in metrics.items():
                ed[key]=value
            
            ed['dir']=cfg
            log_time=''
            for s in cfg.split(os.sep):
                if s.find('___')>=0:
                    log_time=s
                    break
            ed['log_time']=log_time
            
            task=edict_to_pandas(ed)
            tasks=tasks.append(task,ignore_index=True,sort=False)
        else:
            print('cannot find log file for',cfg)
            
    return tasks

def dump(tags=['train/fmeasure','val/fmeasure'],
        rootpath=os.path.expanduser('~/tmp/logs/motion'),
        notes=['motion_fcn','motion_sparse','motion_psp'],
        note_gtags=None,
        descripe=['note','epoch'],
        sort_tags=[],
        invalid_param_list=['dir','log_time','root_path',
                            'test_path', 'train_path', 'val_path'],
        delete_nan=False,
        dump_group=False,
        dump_dir=True,
        recent_log_number=100):
    dir_tags=[tags[1],'dir']
    
    for idx,note in enumerate(notes):
        show_tags=[]
        show_tags=tags.copy()
        tasks=summary(rootpath,tags,note,recent_log_number)
        if tasks.empty:
            print(note,'task is empty')
            continue
        
        if note_gtags is None:
            param_list=[]
            for col in tasks.columns:
                if not isinstance(tasks[col][0],(tuple,list)):
                    if len(set(tasks[col]))>1 and (col not in invalid_param_list) and (col not in tags):
                        param_list.append(col)

            print(note,','.join(param_list))
            group_tags=param_list
        else:
            if idx<len(note_gtags):
                group_tags=note_gtags[idx]
            else:
                group_tags=[]
            
        show_tags+=group_tags
        show_tags+=descripe
        show_tags=list(set(show_tags))
        
        # remove empty dirs
        if delete_nan:
            dirs=tasks[tasks[tags[1]].isna()]['dir'].tolist()
            tasks=tasks[tasks[tags[1]].notna()]
            for dir in dirs:
                print(os.path.dirname(dir))
                os.system('rm -rf {}'.format(os.path.dirname(dir)))

        #print(tasks[show_tags].groupby(group_tags).max().to_string())
        if dump_group:
            if len(group_tags)==0:
                print(group_tags,param_list,tags,invalid_param_list)
            else:
                print(tabulate(tasks[show_tags].groupby(group_tags).mean().sort_values(tags[1]),tablefmt='pipe',headers='keys'))
                print('\n')
                print(tabulate(tasks[[tags[1]]+group_tags].groupby(group_tags).agg([np.mean,np.std,np.max]),tablefmt='pipe',headers='keys'))
                print('\n')
        
        if note=='recent':
            
            sort_tags=['log_time'] if sort_tags is None or len(sort_tags)==0 else sort_tags
            print(tabulate(tasks[show_tags].sort_values(sort_tags),tablefmt='pipe',headers='keys'))
        else:
            print(tabulate(tasks[show_tags].sort_values(sort_tags+[tags[1]]),tablefmt='pipe',headers='keys'))
        print('\n')
        if dump_dir:
            print(tabulate(tasks[dir_tags].sort_values(tags[1]),tablefmt='pipe',headers='keys'))
            print('\n')
    
    return tasks

['/data02/jiaxin15/git/torchseg', '/data02/jiaxin15/git/torchseg', '/data02/jiaxin15/git/torchseg', '/data02/jiaxin15/git/torchseg', '', '/data02/jiaxin15/git/torchseg/notebooks', '/data02/jiaxin15/bin/miniconda3/envs/torch1.0/lib/python36.zip', '/data02/jiaxin15/bin/miniconda3/envs/torch1.0/lib/python3.6', '/data02/jiaxin15/bin/miniconda3/envs/torch1.0/lib/python3.6/lib-dynload', '/data02/jiaxin15/bin/miniconda3/envs/torch1.0/lib/python3.6/site-packages', '/data02/jiaxin15/git/gnu/distiller', '/data02/jiaxin15/bin/miniconda3/envs/torch1.0/lib/python3.6/site-packages/IPython/extensions', '/data02/jiaxin15/.ipython']


In [130]:
tags=['val/fmeasure','val/avg_f']
rootpath=os.path.expanduser('~/tmp/logs/motion')
# attention_type aux_backbone filter_feature
invalid_param_list=['dir','log_time','root_path',
                    'test_path', 'train_path', 'val_path',
                    'use_optical_flow','flow_backbone','ignore_outOfRoi','exception_value']
note_gtags=[['dataset','note'],['dataset','note']]
note_gtags=None
notes=['task274','task275']
tasks=dump(tags=tags,rootpath=rootpath,notes=notes,descripe=['note','epoch'],
     invalid_param_list=invalid_param_list,delete_nan=False,dump_group=True,
     note_gtags=note_gtags)

task274 loss_name
| loss_name   |   val/avg_f |   epoch |   val/fmeasure |
|:------------|------------:|--------:|---------------:|
| ce          |    0.812509 |      50 |       0.849147 |


| loss_name   |   ('val/avg_f', 'mean') |   ('val/avg_f', 'std') |   ('val/avg_f', 'amax') |
|:------------|------------------------:|-----------------------:|------------------------:|
| ce          |                0.812509 |               0.036904 |                0.840887 |


|    |   val/avg_f |   epoch | loss_name   |   val/fmeasure | note    |
|---:|------------:|--------:|:------------|---------------:|:--------|
|  1 |    0.758293 |      50 | ce          |       0.807172 | task274 |
|  0 |    0.818453 |      50 | nan         |       0.850507 | task274 |
|  4 |    0.823531 |      50 | ce          |       0.856835 | task274 |
|  3 |    0.827325 |      50 | ce          |       0.856566 | task274 |
|  2 |    0.840887 |      50 | ce          |       0.876014 | task274 |


|    |   val/avg_f | d

In [122]:
tags=['val/avg_f','val/fmeasure']
rootpath=os.path.expanduser('~/tmp/logs/motion')
notes=['today','recent']
note_gtags=[['dataset','log_time','net_name'],
           ['dataset','log_time','net_name']]
recent_log_number=100
tasks=dump(tags=tags,rootpath=rootpath,notes=notes,note_gtags=note_gtags,sort_tags=['dataset','val/avg_f'],
     descripe=['note','epoch'],delete_nan=False,dump_group=False,recent_log_number=recent_log_number)

|    |   val/avg_f | net_name         | dataset   |   epoch |   val/fmeasure | note        | log_time              |
|---:|------------:|:-----------------|:----------|--------:|---------------:|:------------|:----------------------|
|  7 |    0.758269 | motion_attention | FBMS      |      50 |       0.782075 | task272_h   | 2019-09-16___20-50-57 |
|  2 |    0.762007 | motion_attention | FBMS      |      50 |       0.802429 | task272_s   | 2019-09-16___20-06-09 |
|  0 |    0.762891 | motion_attention | FBMS      |      50 |       0.804034 | task272_n   | 2019-09-16___19-52-37 |
|  5 |    0.773365 | motion_attention | FBMS      |      50 |       0.815193 | task273_gh  | 2019-09-16___20-31-58 |
|  1 |    0.780165 | motion_attention | FBMS      |      50 |       0.820538 | task273_sg  | 2019-09-16___19-54-20 |
|  6 |    0.782842 | motion_attention | FBMS      |      50 |       0.815954 | task272_g   | 2019-09-16___20-35-05 |
|  3 |    0.787474 | motion_attention | FBMS      |      50 |   

In [101]:
notes=['recent']
recent_log_number=1000
tasks=dump(tags=tags,rootpath=rootpath,notes=notes,note_gtags=note_gtags,sort_tags=['dataset','val/avg_f'],
     descripe=['note','epoch'],delete_nan=False,dump_group=False,recent_log_number=recent_log_number)

# tasks=summary(rootpath,tags,note)
print(tags)
sort_tags=['dataset','val/avg_f','val/fmeasure']
show_tags=['note']+sort_tags
tasks_dict={}

threshold={'FBMS':0.8,
         'cdnet2014':0.7,
         'all':0.7,
          'all2':0.7,
          'all3':0.6}
for d,v in threshold.items():
    tasks_dict[d]=tasks[tasks['dataset']==d][tasks[tags[1]]>v]
    print(tabulate(tasks_dict[d][show_tags].sort_values(sort_tags),tablefmt='pipe',headers='keys'))

|     |   val/avg_f | net_name              | dataset   |   epoch |   val/fmeasure | note                          | log_time              |
|----:|------------:|:----------------------|:----------|--------:|---------------:|:------------------------------|:----------------------|
| 840 |    0.483516 | motion_attention      | FBMS      |     100 |     0.554536   | task224_motion_attention      | 2019-08-31___01-42-59 |
| 819 |    0.48565  | motion_panet2         | FBMS      |     100 |     0.552965   | task224_motion_panet2         | 2019-08-30___18-48-02 |
| 888 |    0.498774 | motion_attention      | FBMS      |     100 |     0.257879   | task236_motion_attention      | 2019-09-03___11-44-03 |
| 791 |    0.539461 | motion_panet2         | FBMS      |     100 |     0.586798   | task223_motion_panet2         | 2019-08-29___22-42-53 |
| 865 |    0.54911  | motion_attention      | FBMS      |     100 |     0.483169   | task236_motion_attention      | 2019-09-02___19-54-52 |
| 782 |    0.

|     |   val/fmeasure | dir                                                                                                                        |
|----:|---------------:|:---------------------------------------------------------------------------------------------------------------------------|
| 895 |     0.00161107 | /data02/jiaxin15/tmp/logs/motion/motion_attention/FBMS/task236_motion_attention/2019-09-03___18-19-48/config.txt           |
| 825 |     0.0174732  | /data02/jiaxin15/tmp/logs/motion/motion_attention/FBMS/task227_motion_attention/2019-08-30___20-48-41/config.txt           |
| 870 |     0.181866   | /data02/jiaxin15/tmp/logs/motion/motion_attention/FBMS/task236_motion_attention/2019-09-02___22-56-53/config.txt           |
| 837 |     0.254013   | /data02/jiaxin15/tmp/logs/motion/motion_attention/FBMS/task227_motion_attention/2019-08-31___00-59-08/config.txt           |
| 876 |     0.254537   | /data02/jiaxin15/tmp/logs/motion/motion_attention/FBMS/task236_motion_atten

/data02/jiaxin15/bin/miniconda3/envs/torch1.0/lib/python3.6/site-packages/ipykernel/__main__.py:18: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
